# BIOS 534 - Regression

This note book provides a basic equivalent to the R code presented in class on 3/26. This is not intended to be a one-to-one, lock step reproduction of the R code because, of necessity, the code flow will be different because Python is a different language with its own way of doing things. So it's not possible to make things look the same between the two languages nor should one even try. Each has its strengths. The larger point(s) of the notebook is to show the following:

*   How to Build A Regression Model And Compute RMSE
*   How To Do K Fold Validation Using a Helper Function
*   How To Do K Fold Validation Manually


In [0]:
# Import some needed modules

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


# A Basic Example

So we'll start by building a Regression model on the entire mtcars data frame. We did this with the R example. Ordinarily, you wouldn't do this but this gives you an idea of how you would build the model. We'll worry about train / test and cross validation later in this notebook.

## Read In The Data

Next, we read in the mtcars data frame from the Internet


In [3]:
# Read In The mtcars data frame
url = "https://raw.githubusercontent.com/steviep42/bios534_spring_2020/master/data/mtcars.csv"
dfmt = pd.read_csv(url) 
dfmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 11 columns):
mpg     32 non-null float64
cyl     32 non-null int64
disp    32 non-null float64
hp      32 non-null int64
drat    32 non-null float64
wt      32 non-null float64
qsec    32 non-null float64
vs      32 non-null int64
am      32 non-null int64
gear    32 non-null int64
carb    32 non-null int64
dtypes: float64(5), int64(6)
memory usage: 2.9 KB


## Identify The Target Feature

So we need to determine what the target / outcome feature is and what the predictor features are. We want to predict mpg as a function of weight so we need to index into the data frame to get those values.  

In [5]:
# Identify the Y / outcome feature
Y = dfmt.mpg

# Identify the predictor features. Since we are using just a single column 
# we have to reshape it. Don't know why but that's the way it is

X = np.ravel(dfmt.wt).reshape(-1,1)

# Print them Out
Y



0     21.0
1     21.0
2     22.8
3     21.4
4     18.7
5     18.1
6     14.3
7     24.4
8     22.8
9     19.2
10    17.8
11    16.4
12    17.3
13    15.2
14    10.4
15    10.4
16    14.7
17    32.4
18    30.4
19    33.9
20    21.5
21    15.5
22    15.2
23    13.3
24    19.2
25    27.3
26    26.0
27    30.4
28    15.8
29    19.7
30    15.0
31    21.4
Name: mpg, dtype: float64

## Create The Linear Regression Object

Now we will create a Linear Regression object and start working with that. We'll first build the model using the entire data set itself. Ordinarily, this isn't something that you would do but this is just a first example.


In [8]:
# Initialize a Linear Regression object
model = LinearRegression()

# Fit the model with the training data
model.fit(X,Y)

# Get the coefficeints of the trained model
print('\nCoefficient of model :', model.coef_)

# Get the intercept
print('slope:', model.intercept_)

# R-squared
r_sq = model.score(X, Y)
print('R-Squared:', r_sq.round(3))


Coefficient of model : [-5.34447157]
slope: 37.28512616734204
R-Squared: 0.753


## Making Some Predictions

Look at the predictions using the training data. We are using the same dataset that we trained the model on. 

In [13]:
predict_train = model.predict(X)
print('\nMPG on training data',predict_train) 

# Root Mean Squared Error on training dataset
rmse_train = mean_squared_error(Y,predict_train)**(0.5)
print('\nRMSE on training dataset : ', rmse_train.round(2))



MPG on training data [23.28261065 21.9197704  24.88595212 20.10265006 18.90014396 18.79325453
 18.20536265 20.23626185 20.45004071 18.90014396 18.90014396 15.53312687
 17.3502472  17.08302362  9.22665041  8.29671236  8.71892561 25.52728871
 28.65380458 27.47802083 24.11100374 18.47258623 18.92686632 16.76235533
 16.73563297 26.94357367 25.847957   29.19894068 20.34315128 22.48093991
 18.20536265 22.4274952 ]

RMSE on training dataset :  2.95


# Use a Training / Test Pair

So as we learned in the lecture, a reasonable next step is to create a Train / Test pair which would help provide a better estiamte of out-of-sample error than training purely on the data set. Just as R has some helper functions for this, so does Python.

In [18]:
# Import the train / test module
from sklearn.model_selection import train_test_split

# We'll use an 80/20 split
seed = 7
test_size = .20

# The train_test_split function does the work
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# Check the sizes of the Train Data
X_train.size
Y_train.size


25

## Repeat The Regression

So we'll do the same as before except this time we'll train the model using only the training subset. Then we'll see what the RMSE is on the training data. After that we'll then use it on the test data to see if the RMSE on the test data is similar.

In [29]:
# Initialize another Linear Regression object
model2 = LinearRegression()

# Fit the model with the training data
model2.fit(X_train,Y_train)

# Look at the predictions using the train data
predict_train = model2.predict(X_train)
print('\nMPG on training data',predict_train) 

# Root Mean Squared Error on training dataset
rmse_train = mean_squared_error(Y_train,predict_train)**(0.5)
print('\nRMSE on train dataset : ', rmse_train)


MPG on training data [ 8.76098762 29.74213411 25.98063942 18.75331338 15.74192754 22.80499616
 17.60351151 29.18365891 23.68103568 18.47955103 18.47955103 16.97385811
 20.66964983 19.19133314 22.85974863 20.77915477  9.28113608 20.56014489
 17.00123435 27.97910457 20.42326372 19.21870938 27.43157987 19.19133314
  8.3284431 ]

RMSE on train dataset :  3.2206761356209834


## RMSE On The Test Data

So now let's look at the performance on the test data set. 

In [30]:
# Predict the target feature using the testing dataset
predict_test = model2.predict(X_test)
print('\nMPG on test data',predict_test) 

# Now compute the Root Mean Squared Error on the test dataset
rmse_test = mean_squared_error(Y_test,predict_test)**(0.5)
print('\nRMSE on test dataset : ', rmse_test)



MPG on test data [25.32360978 26.30915424 22.2848477  19.19133314 17.32974916 19.0818282
 24.52969897]

RMSE on test dataset :  1.8045175586043039


# Assisted Cross Fold Validation

We'll use a convenient function from the sklearn library called cross_validate to help us. This is somewhat similar to the train function from the R caret package in that it automates the creation of a series of train / test data splits.


In [0]:
# Import the helper function
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold


# Create a Kfold object - K=4 folds
kfold = KFold(n_splits=4, random_state=7, shuffle=True)

## The Kold Function

The KFold function will manage the creation of the folds, in this case 4 of them.  We get back the indices corresponding to rows of the training data. We also get back the indices corresponding to the holdout or test fold 

In [40]:
for train_index, test_index in kfold.split(X,Y):
      print(train_index)
      print(test_index,"\n")

[ 0  3  4  6  7  8 10 11 12 14 15 17 18 19 21 22 23 24 25 27 28 29 30 31]
[ 1  2  5  9 13 16 20 26] 

[ 1  2  3  4  5  6  7  8  9 10 13 14 15 16 19 20 22 23 24 25 26 28 29 30]
[ 0 11 12 17 18 21 27 31] 

[ 0  1  2  3  4  5  7  9 11 12 13 15 16 17 18 19 20 21 22 23 25 26 27 31]
[ 6  8 10 14 24 28 29 30] 

[ 0  1  2  5  6  8  9 10 11 12 13 14 16 17 18 20 21 24 26 27 28 29 30 31]
[ 3  4  7 15 19 22 23 25] 



## The cross_valdiate function

Next up we will initialize the model and use the cross_validate function to manage the creation of 4 models, one for each fold, and then get a sense of the RMSE for each of the four models. 


In [53]:
from sklearn.metrics import make_scorer

# Create a Linear Regression Object
model3 = LinearRegression()

# We have to designate a scoring metric 
mse = make_scorer(mean_squared_error)

# The cross_validate function handles the execution of the model
cv  = cross_validate(model3,X_train,Y_train,scoring=(mse), cv=kfold)

# Here we print out the rmse of the test / holdout data
print((cv['test_score']**0.5))

# Look at the mean of this RMSE array
print((cv['test_score']**0.5).mean().round(2))

[3.49296896 4.05455881 3.55885475 3.64312596]
3.69


# Manual Cross Fold Validation 

The following is what you might do if you wanted to do the cross fold valdiation on your own without using the convenience of the cross_validate function. This is useful if you want finer grained control over the individual per fold models and perhaps wanted to make predictions on the hold out fold as part of the loop.


## Loop Through The Folds

Here we can create a loop that processes each of the folds. Just to summarize, here is what this process is doing. In reality, this is the same process as in the most recent section just that here we are writing our own loop

1.   We split up the training set into K folds (4 here)
2.   A Model is trained usinf K-1 folds as Training Data
3.   The remaining fold is used as a test data set
4.   Make a prediction on the test data and compute the RMSE
5.   Steps 2 -4 are repeated until all folds have been used as test data 
6.   At the end, return the vector containing all compute RMSEs



In [65]:
# Set up some vectors to collect info

rmse_train_info = np.empty(0)
rmse_test_info = np.empty(0)
ii = 1

# Main processing loop for the folds

for train_index, test_index in kfold.split(X, Y):
  # split data coming from each of the 4 folds
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        
        # Initialize a model
        regress = LinearRegression()
        
        # Fit the Model
        regress.fit(X_train,Y_train)
        
        # Predict the target on the training dataset
        predict_train = regress.predict(X_train)
  
        # Root Mean Squared Error on training dataset
        rmse_train = mean_squared_error(Y_train,predict_train)**(0.5)
        print("Training RMSE for loop",ii,"is:", rmse_train)
       
        # Append the rmse to the rmse_train_info vector
        rmse_train_info = np.append(rmse_train_info,rmse_train)
        
        # Now let's do a prediction on the test data
        predict_test = regress.predict(X_test)
        
        # Root Mean Squared Error on training dataset
        rmse_test = mean_squared_error(Y_test,predict_test)**(0.5)
        print("Test RMSE for loop",ii,"is:", rmse_test,"\n")
       
        # Append the rmse to the rmse_test_info vector
        rmse_test_info = np.append(rmse_test_info,rmse_test)

        ii = ii+1
        

Training RMSE for loop 1 is: 2.9952712670326016
Test RMSE for loop 1 is: 3.1200477195304077 

Training RMSE for loop 2 is: 2.91766901106953
Test RMSE for loop 2 is: 3.2438734462358703 

Training RMSE for loop 3 is: 2.932394228015021
Test RMSE for loop 3 is: 3.1009277330714276 

Training RMSE for loop 4 is: 2.759487962556093
Test RMSE for loop 4 is: 3.5729602245510548 



## Compare the RMSE Outcomes

As part of the loop we captured the RMSE values for the the training and test folds. We can look at the vectors and compute some basic statistics on them.


In [67]:
print("RMSE For Training Folds", rmse_train_info)

print("RMSE For Test Folds",rmse_test_info)


RMSE For Training Folds [2.99527127 2.91766901 2.93239423 2.75948796]
RMSE For Test Folds [3.12004772 3.24387345 3.10092773 3.57296022]


Next what is the mean of these vectors:


In [69]:
print("RMSE Training mean:",rmse_train_info.mean())
print("RMSE Test mean:",rmse_test_info.mean())


RMSE Training mean: 2.9012056171683116
RMSE Test mean: 3.2594522808471904
